Equação de posição:


In [ ]:
class Functions():
    
    def position_equation(self, initial_position:float, velocity:float, time:float) -> float:  #posição
        return initial_position + velocity*time

    def zero_array(self, length: int) -> list:
        
        result_array = []
        iterator = 0 
        
        while iterator < length:
            result_array.append(0)
            iterator += 1
        
        return result_array    
    
    def amax_like(array_to_find_max, axis ):
        return None
    
    def vstack_like():
        return None
    
    def root_function(self, number_to_root, )
    def sqrt_like(self, number_to_sqrt):
        return None

    

functions = Functions()

In [ ]:
def s_asterisco()

In [ ]:
class Vehicle():
    def __init__(self, guided_by, desired_speed, initial_speed, time_between_cars, maximum_acceleration
    , maximum_slowdown, acceleration_exponent, minimum_interval, track_length, time_step, total_time,
    correction_factor, initial_time):
        self.guided_by = guided_by
        self.desired_speed = desired_speed #v0
        self.initial_speed = initial_speed #v_inicial
        self.time_between_cars = time_between_cars #T
        self.maximum_acceleration = maximum_acceleration #a
        self.maximum_slowdown = maximum_slowdown #b
        self.acceleration_exponent = acceleration_exponent #delta
        self.minimum_interval = minimum_interval #s0 in meters
        self.track_length = track_length #fim_da_pista in meters
        self.time_steps = time_step #passos_tempo -> 
        self.total_time = total_time #tempo_total
        self.correction_factor = correction_factor #c
        self.initial_time = initial_time #t_inicial
        self.time_step = (self.total_time - self.initial_time)/self.time_steps
        #self.number_of_cars = number_of_cars
    
    def s_asterisco(self, delta_v,v):
        zeros =  functions.zero_array(length = len(delta_v))
        return self.minimum_interval + np.amax(np.vstack((zeros, v*self.time_between_cars + v*delta_v/(2*functions.sqrt_like(self.maximum_acceleration * self.maximum_slowdown)))), axis = 0)
        #return s0 + np.amax(np.vstack((zeros,v*T + v*delta_v/(2*np.sqrt(a*b)))),axis=0) #Eq. 11.15
    
    def a_IDM(self, v,s,delta_v,params): #IDM
        return self.maximum_acceleration*(1-(v/self.desired_speed)**(self.acceleration_exponent) - (self.s_asterisco(v, delta_v)/self.s)**2) #Eq. 11.14
    
    def a_free_f(self, v):#page 196
        
        comp = (v <= self.desired_speed) # 1 or 0
        
        return (comp*(self.maximum_acceleration*(1-(v/self.desired_speed)**self.acceleration_exponent))
        +(1-comp)*(-self.maximum_slowdown*(1-(self.desired_speed/v)**
        (self.maximum_acceleration*self.acceleration_exponent/self.maximum_slowdown)))) #Eq. 11.22

    
#class IDM(Vehicle):
#class HDM(Vehicle):        
    

    params['passos_tempo'] = 1000 # numero de intervalo de tempo
    params['c'] = 0.99 # fator de correção
    params['t_inicial'] = 0.0 #tempo inicial
    


   
    params['Tr'] = 0.6 
    params['Vs'] = 0.1 
    params['sigma_r'] = 0.01 
    params['sigma_a']= 0.1  
    params['tau_til'] = 20.0 
    params['tau_a_til'] =  1.0 
    